In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
HUGGINGFACE_MODEL_PATH = "beomi/KcELECTRA-small-v2022"
HUGGINGFACE_MODEL_BERT_PATH = "jinmang2/klue-roberta-large-bt-tapt"


model_name = 'KcELECTRA-2022-OOD-test'


batch_size = 8
num_epochs = 2

In [24]:
! pip install transformers
! pip install datasets

In [3]:
#  TPU 초기화
import tensorflow as tf
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)


In [4]:
strategy = tf.distribute.TPUStrategy(resolver)

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tqdm
from transformers import Trainer, TrainingArguments
import re
import pyarrow as pa
from datasets import Dataset
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
from sklearn.model_selection import train_test_split
from datasets import load_metric

## 데이터 불러오기

In [6]:
data = pd.read_csv('/content/drive/MyDrive/dlton/EDA_TRAIN_FINAL_v1.csv')

## 데이터 전처리

In [7]:
labels = {'협박 대화':0, '갈취 대화':1, '직장 내 괴롭힘 대화':2, '기타 괴롭힘 대화':3}

num_labels = len(labels)
data_ = data.copy()

data_['label'] = data_['class'].apply(lambda x: labels[x])

## Tokenizer 및 Mdoel 불러오기

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import ElectraTokenizer, ElectraForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification


tokenizer = AutoTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH)
model = AutoModelForSequenceClassification.from_pretrained(HUGGINGFACE_MODEL_PATH, num_labels=len(labels))


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-small-v2022 and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
len(labels)

4

## 학습을 위한 데이터셋 생성

In [10]:
import pyarrow as pa
from datasets import Dataset

data_ = Dataset(pa.Table.from_pandas(data_))

In [11]:
data_

Dataset({
    features: ['Unnamed: 0', 'class', 'conversation', 'label'],
    num_rows: 4342
})

In [12]:
def transform(data):
  return tokenizer(
      data['conversation'],
      truncation = True,
      padding = 'max_length',
      return_token_type_ids = True,
      )
encoded_data = data_.map(transform, batched=True)

Map:   0%|          | 0/4342 [00:00<?, ? examples/s]

In [13]:
train_dataset, validation_dataset= encoded_data.train_test_split(test_size=0.1).values()

In [14]:
pip install transformers[torch]

In [15]:
metric = load_metric("f1")

def compute_metrics(eval_pred):
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels, average="micro")

output_dir = '/content/drive/MyDrive/dlton'

training_arguments = TrainingArguments(
    output_dir,
    evaluation_strategy="epoch", #evaluation하는 빈도
    learning_rate = 2e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = num_epochs,
    weight_decay = 0.01
)

<ipython-input-15-7dcdc8d4f283>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("f1")


In [16]:
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,No log,1.058225,0.744828
2,1.275800,0.873916,0.822989


TrainOutput(global_step=978, training_loss=1.1085444725364264, metrics={'train_runtime': 8615.0685, 'train_samples_per_second': 0.907, 'train_steps_per_second': 0.114, 'total_flos': 229897411829760.0, 'train_loss': 1.1085444725364264, 'epoch': 2.0})

In [18]:


# Load Test-set
test_file_path = '/content/drive/MyDrive/old/test (1).json'
with open(test_file_path, mode='rt', encoding='utf-8') as f:
    test_dataset = pd.read_json(f)

test_data = test_dataset.transpose()




In [19]:
test_X = Dataset(pa.Table.from_pandas(test_data))

In [20]:
def test_transform(data):
  return tokenizer(
      data['text'],
      truncation = True,
      padding = 'max_length',
      return_token_type_ids = True,
      )

encoded_test_data = test_X.map(test_transform, batched=True)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [21]:
prediction = trainer.predict(encoded_test_data)

In [22]:
len(prediction.predictions)
pred_label = []

for i in range(len(prediction.predictions)):
    pred_label.append((str)(np.argmax(prediction.predictions[i])).zfill(2))

pred_label[:10]

['01', '02', '02', '02', '00', '00', '01', '01', '00', '02']

In [27]:
test_submission = test_data.assign(CLASS=pred_label)
test_submission = test_submission.rename(columns={'CLASS':'class'})

In [28]:
test_submission.drop(['text'], axis=1, inplace=True)

In [29]:
submission_file_path = '/content/drive/MyDrive/dlton/KcELECTRA_small_2epoc_submission.csv'
test_submission.to_csv(submission_file_path, index=True)